In [ ]:
import tensorflow
print(tensorflow.__version__)

In [ ]:
!cp "../data/normal_titles.tx" ./
!cp "../data/normal_queries.txt" ./

In [ ]:
titles_corpus = []
dids = []
didToLine = {}
titles=[]
with open("normal_titles.txt") as f:
    for i,line in enumerate(f):
        line = line.strip()
        did,title = line.split("\t")
        did = int(did)
        words = title.split(" ")
        titles.append(title)
        titles_corpus.append(words)
        dids.append(did)
        didToLine[did] = i

In [ ]:
qidToTopTitle = {}
with open("top_titles2.txt") as f:
    for i,line in enumerate(f):
     
      line = line.strip()
      parts = line.split(maxsplit=1)
      if len(parts)==2:
          qid = int(parts[0])
          title = parts[1]
          qidToTopTitle[qid] = title.lower()

In [ ]:
queries_corpus = []
qids = []
qidToLine = {}
queries = []
with open("normal_queries.txt") as f:
    for line in f:
        line = line.strip()
        qid,title = line.split("\t")
        qid = int(qid)
        words = title.split(" ")
        queries.append(title)
        queries_corpus.append(words)
        qidToLine[qid] = len(qids)
        qids.append(qid)
        

In [ ]:
def getQuery(did):
    global queries
    global qidToLine
    if qid in  qidToLine:
        return queries[qidToLine[qid]]
    return ""
def getTitle(did):
    global titles
    global didToLine
    if did in  didToLine:
        return titles[didToLine[did]]
    return ""
def getTitlesForQuery(qid):
    global titles
    global didToLine
    global allQidToDocs
    rv=[]
    dids = []
    if qid in allQidToDocs:
        for did in allQidToDocs[qid]:
            if did in  didToLine:
                rv.append(titles[didToLine[did]])
                dids.append(did)
    return rv, dids  

In [ ]:
import operator
from collections import defaultdict
import numpy as np
def parseTrainTestFile(train_marks_filename, isTest=False):

    trainDidToLine = {}
    qidToDocs = defaultdict(list)
    
    with open(train_marks_filename) as f:
        i=0
        nlines = 0
        for line in f:
            nlines+=1
        
       
        qids = np.zeros( nlines, dtype=int)
        i=0
        f.seek(0)
        if isTest:
            f.readline()
        for ii,line in enumerate(f):
            if not isTest:
                parts = line.split("\t")
                if len(parts)!=3:
                    continue
                qid, did, mark = parts
                qid, did, mark = int(qid), int(did), int(mark)
            else:
                parts = line.split(",")
                if len(parts)!=2:
                    continue
                qid, did = parts
                mark=-1
                qid, did, mark = int(qid), int(did), int(mark)

            qids[i] = qid
            trainDidToLine[(qid,did)] = i

            qidToDocs[qid].append(did)
            i+=1
    return qids, trainDidToLine, qidToDocs
q_train, trainDidToLine, trainQidToDocs = parseTrainTestFile("train.marks.tsv", 
                                                    isTest=False, 
                                                    )

q_test, testDidToLine, testQidToDocs = parseTrainTestFile("sample.csv", 
                                                    isTest=True, 
                                                    )
def url_normalize(url):
    url = url.strip()
    url = url.replace("https://", "")
    url = url.replace("http://", "")
    if len(url)>1:
        if url[-1] == "/":
            url = url[:-1]
        url = url.lower()
        url = url.replace(":80", "")
        url = url.replace("www.", "")
        url = url.replace("//", "")
    return url
def extract_domain(url):
    global rubrics_list
    url = url.strip()
    #for r in rubrics_list:
        #url = url.replace("," + r + "," , "")    
    #    url = url.replace("," + r, "") 
    url = url.replace("https://", "")
    url = url.replace("http://", "")
   
    url = url.replace("ftp://", "")
    if len(url)>1:
        if url[-1] == "/":
            url = url[:-1]
        url = url.lower()
        
        url = url.replace("s://", "")
        url = url.replace(":80", "")
        url = url.replace("://", "")
        url = url.replace("www.", "")
        url = url.replace("//", "")
        pos = url.find("#")
        if pos>0:
            url = url[0:pos]

        pos = url.find("/")
        if pos>0:
            url = url[0:pos]
    return url
def loadUrls(filename):
    urlToDid = {}
    didToUrl = {}
    domainToDid = defaultdict(list)
    didToDomain = defaultdict(list)
   
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            uid, url = line.split("\t")
            uid = int(uid)
            #urls["http://" + url] = uid
            #urls["http://" + url + "/"] = uid
            urlToDid[url_normalize(url)] = uid
            didToUrl[uid] = url_normalize(url)
            domain = extract_domain(url)
            domainToDid[domain].append(uid)
            didToDomain[uid] = domain
            
    return urlToDid,didToUrl,domainToDid,didToDomain
urlToDid,didToUrl,domainToDid,didToDomain = loadUrls("url.data")
urls = urlToDid

In [ ]:
allQidToDocs = defaultdict(list)
didToQueries = defaultdict(list)
for qid,docs in trainQidToDocs.items():
    allQidToDocs[qid]+=(docs)
    for doc in docs:
        didToQueries[doc].append(qid)
for qid,docs in testQidToDocs.items():
    allQidToDocs[qid]+=(docs)
    for doc in docs:
        didToQueries[doc].append(qid)

In [ ]:
cnt=0
not_matched = 0
nqueries=0

def loadTopDocsFile(filename):
    found_docs = {}
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            qid,fd = line.split(",")
            qid = int(qid)
            fd = int(fd)
            found_docs[qid] = (fd,0.0)
    return found_docs
found_docs = loadTopDocsFile("first_top_docs.txt")
found_domains = loadTopDocsFile("first_top_domains.txt")

In [ ]:
questions = queries

In [ ]:
all_qids = [qid for qid in allQidToDocs]

In [ ]:
len(all_qids)

6311

In [ ]:
!pip3 install tensorflow_text


In [ ]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

# library test from examples
english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]
italian_sentences = ["cane", "I cuccioli sono carini.", "Mi piace fare lunghe passeggiate lungo la spiaggia con il mio cane."]
japanese_sentences = ["犬", "子犬はいいです", "私は犬と一緒にビーチを散歩するのが好きです"]




In [ ]:
def getContentForQid(qid):
    print("Getting content for ", qid)
    dids = []
    contents= []
    try:
        with open("../data/content_by_queries/" + str(qid) + ".txt") as f:
            for line in f:
                parts = line.split("\t")
                if len(parts)>=3:
                    did, title, content = int(parts[0]), parts[1], parts[2]
                    dids.append(did)
                    contents.append(content)
        return contents, dids
    except Exception as e:
        #print(repr(e))
        pass

In [ ]:
embeds = [hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3"),
          hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"),
          hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5"),
          hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3'),
          hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')
         ]


In [ ]:
# library test 
ru_result = en_result = embed(["мама мыла раму"])
ru_result2 = embed(["кабан пришёл домой"])
ru_result3 = embed(["рама помыла ванну"])
print(np.inner(ru_result, ru_result2), np.inner(ru_result2, ru_result3), np.inner(ru_result, ru_result3))

[[0.28125972]] [[0.25637293]] [[0.68262917]]


In [ ]:
for eid, embed in enumerate(embeds):
    with open("USE_out2_" + str(eid) + ".txt", "w") as f:
      flag=True
      for qid in all_qids:
        try:
          if(flag):
            qtitles,dids  = getTitlesForQuery(qid)
            query = getQuery(qid)
            top_score = np.zeros(len(dids))
            top_score2 = np.zeros(len(dids))
            if eid<=2:
                title_emb = embed(qtitles)
                query_emb = embed([query])[0]
            else:
                response_contexts,content_dids = getContentForQid(qid)
                title_emb = embed.signatures['question_encoder'](tf.constant([query]))
                query_emb = embed.signatures['response_encoder'](input=tf.constant(qtitles), context=tf.constant(response_contexts))
                
           
                
            if qid in found_docs:
                top_did = found_docs[qid][0]
                top_title = getTitle(top_did)  
                if eid<=2:
                    top_emb = embed([top_title])[0]
               
                    top_score = np.inner(top_emb, title_emb)
            if qid in qidToTopTitle:
                top_title = qidToTopTitle[qid] 
                if eid<=2:
                    top_emb = embed([top_title])[0]
                
                    top_emb = embed.signatures['question_encoder'](tf.constant([top_title]))
                    top_score2 = np.inner(top_emb, title_emb)
                else:
                    pass
                #
            

            score = np.inner(query_emb, title_emb)

            for i,s in enumerate(score):
              f.write(str(qid) + "," + str(dids[i]) + "," + str(s) + "," + str(top_score[i]) + "," + str(top_score2[i]) + "\n")
            if qid%100==0:
                if qid==1200 :
                    flag=True
                if(flag):
                    print(qid, score)
        except Exception as e:
            print(repr(e), qid)

In [ ]:
import h2o4gpu as sklearn

In [ ]:
def getContentForQid(qid):
    print("Getting content for ", qid)
    dids = []
    contents= []
    try:
        with open("../data/content_by_queries/" + str(qid) + ".txt") as f:
            for line in f:
                parts = line.split("\t")
                if len(parts)>=3:
                    did, title, content = int(parts[0]), parts[1], parts[2]
                    dids.append(did)
                    contents.append(content)
        return contents, dids
    except Exception as e:
        #print(repr(e))
        pass

In [ ]:
!mkdir ../data/res_queries/

In [ ]:
import os
import h2o4gpu
import h2o4gpu as sklearn
from h2o4gpu.feature_extraction.text import TfidfVectorizer
from h2o4gpu.metrics.pairwise import cosine_similarity
def processQuery(qid):
  try:
    path = "../data/res_queries/" + str(qid) + ".txt"
    
    with open(path, "w") as f:
      
        #ts, dids = getTitlesForQuery(qid)
    
        contents, contentDids = getContentForQid(qid)
        dids = contentDids
        ts  = contents
        thisdidtoline = {}
        for ind,tmp in enumerate(dids):
            thisdidtoline[tmp] = ind
        query = getQuery(qid)

        
        
        TextVectorizerWord  = TfidfVectorizer(analyzer="word", ngram_range=(1,3))
        
        contentWord = TextVectorizerWord.fit_transform(contents)
        contentWordQuery = TextVectorizerWord.transform([query])
        contentQuerySim = cosine_similarity(contentWord, contentWordQuery)
        
        
        TextVectorizerNgram = TfidfVectorizer(analyzer="char", ngram_range=(3,12))
        contentNgram = TextVectorizerNgram.fit_transform(contents)
        contentNgramQuery = TextVectorizerNgram.transform([query])
        contentNgramSim = cosine_similarity(contentNgram, contentNgramQuery)
        
       
        sqsim = np.zeros((len(dids),1))
        fcsim = np.zeros((len(dids),1))
        if qid in found_docs:
            fdid,score = found_docs[qid]
            if fdid in didToLine:
                t1 = getTitle(fdid)
                #tidf = vectorizer.fit_transform(ts)
                sqidf = TextVectorizerNgram.transform([t1])
                sqsim = cosine_similarity(contentNgram, sqidf)
                
            if fdid in thisdidtoline:
                line = thisdidtoline[fdid]
                fcontent = contents[line]
                fcidf = TextVectorizerWord.transform([fcontent])
                fcsim = cosine_similarity(contentWord, fcidf)
       
        sqsimd = np.zeros((len(dids),1))
        fcsimd = np.zeros((len(dids),1))
        
        if qid in found_domains:
            fdid,score = found_domains[qid]
            if fdid in didToLine:
                t1 = getTitle(fdid)
               
                sqidf = TextVectorizerNgram.transform([t1])
                sqsimd = cosine_similarity(contentNgram, sqidf)
                
            if fdid in thisdidtoline:
                line = thisdidtoline[fdid]
                fcontent = contents[line]
                fcidf = TextVectorizerWord.transform([fcontent])
                fcsimd = cosine_similarity(contentWord, fcidf)
       
     
       
        for i in range(len(ts)):
            
            f.write(str(qid) + "," + str(dids[i]) + "," + str(contentQuerySim[i,0]) + "," + str(contentNgramSim[i,0]) + "," + str(fcsim[i,0]) + "," + str(sqsim[i,0]) + "," + str(fcsimd[i,0]) + "," + str(sqsimd[i,0])  + str("\n") )
            f.flush()
        
  except Exception as e:
    #print(repr(e))
    pass

In [ ]:
all_qids = [qid for qid in allQidToDocs]
import random
random.shuffle(all_qids)

from multiprocessing import Pool
p = Pool(32)
p.map(processQuery, all_qids)

In [ ]:
from h2o4gpu.feature_extraction.text import TfidfVectorizer
from h2o4gpu.metrics.pairwise import cosine_similarity
from multiprocessing import Pool


In [ ]:
!cat "../data/res_queries/*.txt" >> ../data/tfidf_features.txt

In [4]:
def mergeFiles(file1, file2, fileout):
    a = {}
    with open(file1) as f:
      for line in f:
        line = line.strip()
        parts = line.split(",")
        if len(parts)>2:
          qid,did = int(parts[0]), int(parts[1])
          data = list(map(float,(parts[2:])))
          alen = len(data)
          a[(qid,did)] = data
    b = {}
    with open(file2) as g:
        for line in g:
          line = line.strip()
          parts = line.split(",")
          if len(parts)>2:
            qid,did = int(parts[0]), int(parts[1])
            data = list(map(float,(parts[2:])))
            blen = len(data)
            b[(qid,did)] = data
    rd = {}
    for (qid,did), data in a.items():
       if  (qid,did) not in b:
          b[ (qid,did)] = [0.0] * blen
    for (qid,did), data in b.items():
       if (qid,did) not in a:
          a[(qid,did)] = [0.0] * alen
    for (qid,did), data in b.items():
      rd[(qid,did)] = a[(qid,did)]
      rd[(qid,did)] += data
    with open(fileout, "w") as h:
        for (qid,did), data in rd.items():
          data = list(map(str,(data)))
          h.write(str(qid) +"," + str(did) +"," + ','.join(data) + "\n" )

In [ ]:
!cp ../data/tfidf_features.txt ./USE_tfidf_features.txt
for i in range(len(embeds)):
    mergeFiles("USE_out2_" + str(i) + ".txt", "USE_tfidf_features.txt", "USE_tfidf_features.txt")